## Workspace Management demo

This notebook presents examples of how to manage and use team workspaces!

In [1]:
import jwt
import requests
import datetime
import boto3
import urllib3
import time
urllib3.disable_warnings()

In [2]:
realm = "eoepca"
base_domain = "apx.develop.eoepca.org"
keycloak_endpoint = f"https://iam-auth.{base_domain}"
workspace_api_endpoint = f'https://workspace-api.{base_domain}/workspaces'
token_endpoint = f"{keycloak_endpoint}/realms/{realm}/protocol/openid-connect/token"
minio_endpoint = "https://minio.develop.eoepca.org"

### token handling

In [3]:
def iam_token(username, password):
    headers = {
        "Cache-Control": "no-cache",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "scope": "roles",
        "grant_type": "password",
        "username": username,
        "password": password,
        "client_id": "demo",
        "client_secret": "demo"
    }    
    response = requests.post(token_endpoint, headers=headers, data=data)
    if response.ok:
        return response.json()["access_token"]
    else:
        print(response)
        return None

**Important**: We expect `example-user-1`, `example-user-2` and `example-admin` to exist in Keycloak!

In [4]:
#the following example users are expected to exist
token_user1 = iam_token("example-user-1", "changeme")
decoded_token_user1 = jwt.decode(token_user1, options={"verify_signature": False})
print(f"\n{decoded_token_user1}")
token_user2 = iam_token("example-user-2", "changeme")
decoded_token_user2 = jwt.decode(token_user2, options={"verify_signature": False})
print(f"\n{decoded_token_user2}")
token_admin = iam_token("example-admin", "changeme")
decoded_token_admin = jwt.decode(token_admin, options={"verify_signature": False})
print(f"\n{decoded_token_admin}")


{'exp': 1744036837, 'iat': 1744036537, 'jti': '6dc70454-c18e-43c4-b12f-df8e807b2678', 'iss': 'https://iam-auth.apx.develop.eoepca.org/realms/eoepca', 'aud': ['eoapi', 'resource_registration', 'ws-example-team-1', 'account', 'ws-example-team-2'], 'sub': '0965f986-de8e-43fa-ba1c-36d520eefe6f', 'typ': 'Bearer', 'azp': 'demo', 'sid': '2b668883-fc7d-45b0-8760-eb6ba1a798cc', 'acr': '1', 'realm_access': {'roles': ['offline_access', 'default-roles-eoepca', 'uma_authorization']}, 'resource_access': {'eoapi': {'roles': ['stac_editor']}, 'resource_registration': {'roles': ['records_editor']}, 'ws-example-team-1': {'roles': ['ws_access']}, 'account': {'roles': ['manage-account', 'view-consent', 'manage-account-links', 'manage-consent', 'view-profile']}, 'ws-example-team-2': {'roles': ['ws_access']}}, 'scope': 'profile email', 'email_verified': False, 'preferred_username': 'example-user-1'}

{'exp': 1744036837, 'iat': 1744036537, 'jti': 'eab7a815-5be8-45bf-878a-51df74db9b24', 'iss': 'https://iam-a

In [5]:
print(decoded_token_user1["aud"])
print(decoded_token_user1["resource_access"])

['eoapi', 'resource_registration', 'ws-example-team-1', 'account', 'ws-example-team-2']
{'eoapi': {'roles': ['stac_editor']}, 'resource_registration': {'roles': ['records_editor']}, 'ws-example-team-1': {'roles': ['ws_access']}, 'account': {'roles': ['manage-account', 'view-consent', 'manage-account-links', 'manage-consent', 'view-profile']}, 'ws-example-team-2': {'roles': ['ws_access']}}


### review existing workspaces

In [6]:
ws_user1 = [entry for entry in decoded_token_user1["aud"] if entry.startswith('ws-')]
print(f"user1: {ws_user1}")
ws_user2 = [entry for entry in decoded_token_user2["aud"] if entry.startswith('ws-')]
print(f"user2: {ws_user2}")

user1: ['ws-example-team-1', 'ws-example-team-2']
user2: []


In [7]:
def access_ws(ws_name, token):
    headers = {
        'Authorization': 'bearer ' + token
    }
    url = f"{workspace_api_endpoint}/{ws_name}"
    print(f"HTTP GET {url}")
    response = requests.get(url, headers=headers)
    print(response)
    #print(response.text)
    return response

In [8]:
#manually add/remove users in keycloak and check access here
access_ws("ws-example-team-1", token_user1)

HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-team-1
<Response [200]>


<Response [200]>

### workspace creation

In [9]:
preferred_ws_name = f"example-{datetime.datetime.now().timestamp():.0f}"
preferred_ws_name

'example-1744036538'

In [10]:
response = requests.post(
    workspace_api_endpoint,
    headers={
        'Authorization': 'Bearer ' + token_admin
    },
    json={
        "preferred_name": preferred_ws_name,
        "default_owner": "example-user-1"
    }
)
response.raise_for_status()
actual_workspace_name = response.json()['name']
ws_name = response.json()["name"]
print(f"created workspace '{ws_name}'")

created workspace 'ws-example-1744036538'


In [11]:
while True:
    time.sleep(10)
    response = access_ws(ws_name, iam_token("example-user-1", "changeme"))
    if response.status_code == 200:
        try:
            workspace_data = response.json()
            if workspace_data.get("status") == "ready":
                print("ready")
                break
        except ValueError:
            print("not ready yet")
    
    print("...retrying")
    

HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744036538
<Response [200]>
...retrying
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744036538
<Response [200]>
...retrying
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744036538
<Response [200]>
...retrying
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744036538
<Response [200]>
...retrying
HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744036538
<Response [200]>
ready


### workspace permissions

note: example-user-1 is workspace owner (see creation above) an can access!

In [12]:
response1 = access_ws(ws_name, iam_token("example-user-1", "changeme"))
assert response1.status_code == 200

HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744036538
<Response [200]>


note: example-user-2 can't access yet!

In [13]:
response2 = access_ws(ws_name, iam_token("example-user-2", "changeme"))
assert response2.status_code == 403

HTTP GET https://workspace-api.apx.develop.eoepca.org/workspaces/ws-example-1744036538
<Response [403]>


### workspace vcluster (k8s) access 

In [14]:
kubeconfig = workspace_data["runtime"]["envs"]["KUBECONFIG"]

In [15]:
from kubernetes import client, config
from tempfile import NamedTemporaryFile

with NamedTemporaryFile(mode='w+', delete=False) as temp_kubeconfig:
    temp_kubeconfig.write(kubeconfig.replace("\\n", "\n"))
    temp_kubeconfig.flush()

    config.load_kube_config(config_file=temp_kubeconfig.name)
    configuration = client.Configuration.get_default_copy()
    configuration.verify_ssl = False # TBD
    configuration.assert_hostname = False # TBD
    api_client = client.ApiClient(configuration)

In [16]:
v1 = client.CoreV1Api(api_client)
all_pods = v1.list_pod_for_all_namespaces()
for pod in all_pods.items:
    print(f"{pod.metadata.namespace} / {pod.metadata.name}")

kube-system / coredns-668c87c5d8-4nxxc


### workspace s3 access 

In [17]:
bucket_name = workspace_data["storage"]["credentials"]["bucketname"]
s3_access = workspace_data["storage"]["credentials"]["access"]
s3_secret = workspace_data["storage"]["credentials"]["secret"]

In [18]:
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=minio_endpoint)

In [19]:
path = 'application-package/s-expression/s-expression-0_0_2.cwl'

In [20]:
object = s3resource.Object(bucket_name, path)
result = object.put(Body=open('../data/s-expression-cwl.cwl', 'rb'))
res = result.get('ResponseMetadata')
if res.get('HTTPStatusCode') == 200:
    print('Application package uploaded successfully')
else:
    print('Application package not uploaded')

Application package uploaded successfully


In [21]:
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(f"Key: {obj.key}, Size: {obj.size} bytes")

Key: application-package/s-expression/s-expression-0_0_2.cwl, Size: 2302 bytes


### workspace ui with storage layer

In [22]:
print(f"You can now open the Workspace UI at https://{ws_name}.{base_domain} and login with example-user-1")

You can now open the Workspace UI at https://ws-example-1744036538.apx.develop.eoepca.org and login with example-user-1


### workspace deletion

In [23]:
result = object.delete()

In [24]:
response = requests.delete(
    f"{workspace_api_endpoint}/{ws_name}",
    headers={
        'Authorization': 'Bearer ' + iam_token("example-admin", "changeme")
    }
)
response.raise_for_status()
print(f"deleted workspace '{ws_name}'")

deleted workspace 'ws-example-1744036538'
